# Cleaning

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [2]:
data = pd.read_csv('/kaggle/input/toxic-message-classifier-dataset/train_cleaned.csv')
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_words
0,0000997932d777bf,explanation\n edits made username hardcore ...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '..."
1,000103f0d9cfb60f,daww matches background colour im seemingly ...,0,0,0,0,0,0,"['daww', 'matches', 'background', 'colour', 'i..."
2,000113f07ec002fd,hey man im really trying edit war guy c...,0,0,0,0,0,0,"['hey', 'man', 'im', 'really', 'trying', 'edit..."
3,0001b41b1c6bb37e,cant make real suggestions improvement won...,0,0,0,0,0,0,"['cant', 'make', 'real', 'suggestions', 'impro..."
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page', ..."


# Tokenizing Comments using TextVectorization

In [3]:
# finding the maximum number of words present in any given comment
maxlen = 0
longest_comment = ""
for comment in data['comment_text']:
    length = len(comment)
    if (length > maxlen):
        longest_comment = comment
    maxlen = max(maxlen, length)

print("Number of characters in the longest comment is", maxlen)
print("Number of words in the longest comment is",
      len(longest_comment.split(" ")) + 1)

Number of characters in the longest comment is 5000
Number of words in the longest comment is 456


In [4]:
import tensorflow as tf

vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(

    max_tokens=None,

    # this is greater than the max words any comment has (774)
    # the remaning spots in the output would be padded by 0s
    output_sequence_length=800,

    # converets to lowercase and skips all the punctuation
    standardize="lower_and_strip_punctuation",

    # the tokens will be split at whitespaces
    split="whitespace",

    # each of the tokens is represented as an integer
    output_mode="int",
)

In [5]:
numpyArray = data[data.columns[1]].to_numpy()
vectorize_layer.adapt(numpyArray)

In [6]:
# testing
vectorize_layer("hello, world!")

<tf.Tensor: shape=(800,), dtype=int64, numpy=
array([185, 161,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,  

# Creating a Tokenizer

### Gets all the words into one string

In [9]:
# Creates the tokenizer class
tokenizer = keras.preprocessing.text.Tokenizer()

# Combines all the words into one singular string
allWordsString = " ".join(data.head(50)["comment_text"].tolist())
allWordsList = allWordsString.split(r"\\s+")

# Updates the tokenizer with the string of all words
tokenizer.fit_on_texts(allWordsList)

# Prints word dictionary
# print(tokenizer.word_index)

# Prints length of word dictionary
print(len(tokenizer.word_index))

# Converts text to numbers
print(tokenizer.texts_to_sequences(["page", "im", "use", "mussolini"]))

1077
[[1], [2], [3], [1077]]


# Text Vectorization Layer
### Following [https://www.tensorflow.org/text/tutorials/text_classification_rnn](http://)

In [12]:
NUM_ROWS = 10000

MAX_LENGTH = None
encoder = tf.keras.layers.TextVectorization(output_sequence_length=MAX_LENGTH)
encoder.adapt(data.head(NUM_ROWS)["comment_text"].tolist())

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'article', 'page', 'wikipedia', 'would', 'talk',
       'like', 'one', 'please', 'dont', 'see', 'also', 'im', 'know',
       'think', 'edit', 'people', 'use', 'articles'], dtype='<U322')

### Testing the encoder

Encoder removes punctuation and whitespace and forces lowercase so half the cleaning we did was useless

In [13]:
commentsToEncode = data.head(3)["comment_text"]
print(commentsToEncode)

encodedComments = encoder(commentsToEncode).numpy()
print(encodedComments)

for comment in encodedComments:
    print(" ".join(vocab[comment]))

0    explanation\n  edits made   username hardcore ...
1    daww  matches  background colour im seemingly ...
2    hey man im really  trying  edit war     guy  c...
Name: comment_text, dtype: object
[[  530    53    41   437  4205 14587   257   208  2022  9650  6733  4903
   2845    44  1016 11400  2690     9    10   157   357     6     3    56
     13  3853]
 [35388  2050  1647  6725    13  3256  2365    26     6   977    95     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0]
 [  387   349    13    49   169    16   218   524  1606   406   367    30
    399    53   270     6     3   126   341  2062   506   391     0     0
      0     0]]
explanation edits made username hardcore metallica fan reverted werent vandalisms closure gas voted new york dolls fac please dont remove template talk page since im retired
daww matches background colour im seemingly stuck thanks talk january utc               
hey man im really trying edit war guy constantly

### Building the model

In [14]:
# Sets random seed so results are identical every time
SEED = 1
tf.random.set_seed(SEED)
np.random.seed(SEED)

from tensorflow.keras.callbacks import EarlyStopping

epochs = 5
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
embedding_size = 512
lstm_size_1 = 128
conv_size_2 = 64

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=embedding_size,
        mask_zero=True,
        embeddings_regularizer=tf.keras.regularizers.L2(0.001)
    ),
    tf.keras.layers.Conv1D(128, 5, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(conv_size_2, 5, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_size_1, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

# Add early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])


### Training the model

In [15]:
binaryDf = data.head(NUM_ROWS)[["comment_text", "toxic"]]

split_cutoff = int(0.8 * NUM_ROWS)
training_data = binaryDf.iloc[:split_cutoff]
validation_data = binaryDf.iloc[split_cutoff:]

training_target = training_data.pop("toxic")
validation_target = validation_data.pop("toxic")

## Sample Weight = 2

In [17]:
sample_weight = np.ones(shape=(len(training_target),))
sample_weight[training_target == 1] = 2.0

callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=7)
history = model.fit(training_data, training_target, epochs=10, validation_data=(validation_data, validation_target), callbacks=[callback], sample_weight=sample_weight)

Epoch 1/10


/opt/conda/lib/python3.7/site-packages/keras/backend.py:5677: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits, "Sigmoid", "binary_crossentropy"


249/250 [============================>.] - ETA: 0s - loss: 9.5216 - accuracy: 0.9032

/opt/conda/lib/python3.7/site-packages/keras/backend.py:5677: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits, "Sigmoid", "binary_crossentropy"


250/250 [==============================] - 52s 123ms/step - loss: 9.5010 - accuracy: 0.9030 - val_loss: 3.9764 - val_accuracy: 0.9120
Epoch 2/10
250/250 [==============================] - 11s 45ms/step - loss: 2.3008 - accuracy: 0.9043 - val_loss: 1.0307 - val_accuracy: 0.9120
Epoch 3/10
250/250 [==============================] - 10s 39ms/step - loss: 0.8538 - accuracy: 0.9061 - val_loss: 0.4802 - val_accuracy: 0.9290
Epoch 4/10
250/250 [==============================] - 10s 39ms/step - loss: 0.4595 - accuracy: 0.9391 - val_loss: 0.3721 - val_accuracy: 0.9445
Epoch 5/10
250/250 [==============================] - 9s 35ms/step - loss: 0.2890 - accuracy: 0.9796 - val_loss: 0.3662 - val_accuracy: 0.9370
Epoch 6/10
250/250 [==============================] - 9s 38ms/step - loss: 0.2092 - accuracy: 0.9921 - val_loss: 0.4043 - val_accuracy: 0.9265
Epoch 7/10
250/250 [==============================] - 10s 40ms/step - loss: 0.1717 - accuracy: 0.9961 - val_loss: 0.5494 - val_accuracy: 0.9070
Epoc

In [18]:
test_data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test.csv")
test_labels = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test_labels.csv")

test_labels = test_labels.loc[test_labels["toxic"] >= 0]
merged_df = test_labels.merge(test_data, left_on="id", right_on="id")

In [19]:
# Tests all rows with a value of 0 or 1

test_df = merged_df[["comment_text", "toxic"]]
testTarget = test_df.pop("toxic")
model.evaluate(test_df, testTarget)

2000/2000 [==============================] - 48s 24ms/step - loss: 0.8526 - accuracy: 0.8040


[0.8525853753089905, 0.8039951324462891]